In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

In [2]:
fp = './Copy of IFAW Mass Stranding Data 1999-2014.xlsx'
df = pd.read_excel(fp, index_col=0, parse_cols="C,F:I", parse_dates=True)

In [3]:
df.rename(columns={'Number of Animals Stranded': "stranded",
                   "Number of Additional Animals Herded" : "herded", 
                  "LATITUDE_DEGREE": "lat", "LONGITUDE_DEGREE": "lon"},inplace=True)

In [4]:
df.index.name= 'date'

In [5]:
df.head()

,stranded,herded,lat,lon
date,,,,
2000-04-09,2.0,0.0,41.930667,70.069667
2000-04-11,2.0,0.0,41.933333,70.024167
2000-04-11,2.0,0.0,41.933333,70.024167
2000-08-14,NaN,NaN,41.806833,70.002667
2000-08-26,6.0,20.0,41.928167,70.066833


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 231 entries, 2000-04-09 to 1999-12-15
Data columns (total 4 columns):
stranded    160 non-null float64
herded      157 non-null float64
lat         231 non-null float64
lon         231 non-null float64
dtypes: float64(4)
memory usage: 14.0 KB


In [9]:
df.drop_duplicates().dropna().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 155 entries, 2000-04-09 to 1999-12-15
Data columns (total 4 columns):
stranded    155 non-null float64
herded      155 non-null float64
lat         155 non-null float64
lon         155 non-null float64
dtypes: float64(4)
memory usage: 6.1 KB


In [17]:
df.drop_duplicates(inplace=True)

In [21]:
df.dropna(how='all', subset=['stranded', 'herded'], inplace=True)

In [26]:
df.loc[df.stranded.isnull()]

,stranded,herded,lat,lon
date,,,,
2009-12-07,NaN,0.0,41.792267,70.0197


In [27]:
df.loc['2009-12-07', :]

,stranded,herded,lat,lon
date,,,,
2009-12-07,3.0,NaN,41.775783,70.0553
2009-12-07,NaN,0.0,41.792267,70.0197


In [36]:
df.loc[df.index.duplicated(keep=False), 'lon'] = df.loc[df.index.duplicated(keep=False)].lon.abs()

In [37]:
df.loc[df.index.duplicated(keep=False),:]

,stranded,herded,lat,lon
date,,,,
2008-12-25,5.0,0.0,41.784000,70.033100
2008-12-25,3.0,0.0,41.920500,70.032167
2009-12-07,3.0,NaN,41.775783,70.055300
2009-12-07,NaN,0.0,41.792267,70.019700
2012-02-09,5.0,0.0,41.727200,70.303100
2012-02-09,3.0,0.0,41.892300,70.008500


In [42]:
df.replace(np.NaN, 0.0, inplace=True)

In [43]:
df.loc[df.index.duplicated(keep=False),:]

,stranded,herded,lat,lon
date,,,,
2008-12-25,5.0,0.0,41.784000,70.033100
2008-12-25,3.0,0.0,41.920500,70.032167
2009-12-07,3.0,0.0,41.775783,70.055300
2009-12-07,0.0,0.0,41.792267,70.019700
2012-02-09,5.0,0.0,41.727200,70.303100
2012-02-09,3.0,0.0,41.892300,70.008500


In [28]:
df.to_pickle('./dfStranded.pkl')

Aggregating daily strandings, I take the mean of lat/lons for the corresponding coordinates.

In [87]:
dfAgg = pd.DataFrame([{'date': k, 'stranded': v.stranded.sum(), 'herded': v.herded.sum(),
              'lat': v.lat.mean(), 'lon': v.lon.mean()}
              for k, v in df.groupby(df.index)], columns=['date','stranded','herded', 'lat','lon'])

In [88]:
dfAgg.set_index('date', inplace=True)

Reindex to get continuous daily time series

In [92]:
dfAggRidx = dfAgg.reindex(pd.date_range(dfAgg.index[0], dfAgg.index[-1]).tolist())
dfAggRidx.fillna(0.0, inplace=True)

In [93]:
dfAggRidx.head(3)

,stranded,herded,lat,lon
date,,,,
1999-03-07,6.0,0.0,41.916667,70.033333
1999-03-08,0.0,0.0,0.000000,0.000000
1999-03-09,0.0,0.0,0.000000,0.000000


In [94]:
dfAggRidx.to_pickle('./dfStrandedReindexed.pkl')